In [1]:
from EMDA import EMDA, __version__
print("EMDA version is:", __version__)

from numpy import average

import matplotlib.pyplot as plt

/Users/mikicanyelles/miniconda3/envs/emda-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


EMDA version is: 0.2.0a2


# Load parameters and trajectory <a class="anchor" id="load"></a>

In [2]:
parameters = 'parameters.prmtop' # AMBER parameters file
trajectory = 'trajectory.nc'     # AMBER NetCDF coordinates (10 frames)

In [3]:
emda = EMDA(parameters=parameters, trajectory=trajectory)

Trajectory has been loaded!


# Select the atoms to be used for the analysis <a class="anchor" id="select"></a>

In [4]:
emda.select('C10', 'C10', sel_type='at_name')
emda.select('C11', 'C11', sel_type='at_name')
emda.select('C12', 'C12', sel_type='at_name')
emda.select('C13', 'C13', sel_type='at_name')
emda.select('C14', 'C14', sel_type='at_name')

emda.select('H12', ['H12A', 'H12B'], sel_type='at_name')
emda.select('cof', 10597, sel_type='at_num')

emda.select('COO', [10599, 10600, 10601], sel_type='at_num')

subs = emda.universe.select_atoms('resid 666')

In [5]:
print(emda.selections['C12'])

<AtomGroup [<Atom 10626: C12 of type c3 of resname DHA, resid 666 and segid SYSTEM>]>


# Analysis of the trajectory <a class="anchor" id="trajanalysis"></a>

In [6]:
emda.add_distance('dist_H12', 'cof', 'H12')
emda.add_angle('angle_C11C12C13', 'C11', 'C12', 'C13')
emda.add_dihedral('dihe_C10C11C13C14', 'C10', 'C11', 'C13', 'C14', domain=180)
emda.add_contacts('contacts_COO', 'COO', sel_env=5, interactions='all', include_WAT=True, measure_distances=True)
#emda.add_contacts('contacts_prot', 'protein', sel_env=5)
#emda.add_pKa('pka', excluded_ions=['Na+'], keep_pdb=True, keep_pka=True)
emda.add_RMSD('RMSD_subs', subs)

## Run the measurements

In [7]:
print(emda.measures)

{'dist_H12': Measure dataclass with:
	Name:   dist_H12
	Type:   distance
	Sel:    [<AtomGroup with 1 atom>, <AtomGroup with 2 atoms>]
	Status: Not calculated
, 'angle_C11C12C13': Measure dataclass with:
	Name:   angle_C11C12C13
	Type:   angle
	Sel:    [<AtomGroup with 1 atom>, <AtomGroup with 1 atom>, <AtomGroup with 1 atom>]
	Status: Not calculated
, 'dihe_C10C11C13C14': Measure dataclass with:
	Name:   dihe_C10C11C13C14
	Type:   dihedral
	Sel:    [<AtomGroup with 1 atom>, <AtomGroup with 1 atom>, <AtomGroup with 1 atom>, <AtomGroup with 1 atom>]
	Status: Not calculated
, 'contacts_COO': Measure dataclass with:
	Name:   contacts_COO
	Type:   contacts
	Sel:    [<AtomGroup with 3 atoms>, <AtomGroup with 77 atoms, with selection 'around 5 group select' on the entire Universe.>]
	Status: Not calculated
, 'RMSD_subs': Measure dataclass with:
	Name:   RMSD_subs
	Type:   RMSD
	Sel:    [<AtomGroup with 55 atoms>]
	Status: Not calculated
}


In [8]:
emda.run()

Measuring:   0%|          | 0/10 [00:00<?, ?Frame/s]

Measuring: 100%|██████████| 10/10 [00:00<00:00, 266.94Frame/s]


In [9]:
emda.analyse_value('dist_H12_bool', 'dist_H12', 5)

In [10]:
print('angle_C11C12C13 average is:', average(emda.measures['angle_C11C12C13'].result))
print('angle_C11C12C13 min is:', min(emda.measures['angle_C11C12C13'].result))
print('angle_C11C12C13 max is:', max(emda.measures['angle_C11C12C13'].result))

angle_C11C12C13 average is: 112.20870900390655
angle_C11C12C13 min is: 107.89308253365704
angle_C11C12C13 max is: 118.05900472297759


In [11]:
emda.analyse_value('angle_C11C12C13_bool', 'angle_C11C12C13', 110, 5, mode='tol')

In [12]:
emda.measures['angle_C11C12C13'].result

[112.02202569860975,
 111.8937126966033,
 111.26140570617383,
 112.30627552711508,
 107.89308253365704,
 116.13395578048454,
 108.0022472216121,
 110.32166887755017,
 118.05900472297759,
 114.19371127428218]

In [13]:
emda.analyses['angle_C11C12C13_bool'].type

'value'

In [14]:
print(emda.analyses)

{'dist_H12_bool': Analysis dataclass with:
	Name:   dist_H12_bool
	Type:   value
	Related mesure:    dist_H12
	Status: Calculated
, 'angle_C11C12C13_bool': Analysis dataclass with:
	Name:   angle_C11C12C13_bool
	Type:   value
	Related mesure:    angle_C11C12C13
	Status: Calculated
}


In [15]:
emda.analyse_NACs('dist_angle_NACs', ['dist_H12_bool', 'angle_C11C12C13_bool'], inverse=False)


In [17]:
emda.analyse_NACs('dist_angle_NACs_inverse', ['dist_H12_bool', 'angle_C11C12C13_bool'], inverse=['angle_C11C12C13_bool'])


In [19]:
print('dist_H12_bool:\t\t\t', emda.analyses['dist_H12_bool'].result)
print('angle_C11C12C13_bool:\t\t', emda.analyses['angle_C11C12C13_bool'].result)
print('dist_angle_NACs:\t\t', emda.analyses['dist_angle_NACs'].result)
print('dist_angle_NACs_inverse:\t', emda.analyses['dist_angle_NACs_inverse'].result)

dist_H12_bool:			 [True, True, True, True, True, True, True, True, True, True]
angle_C11C12C13_bool:		 [True, True, True, True, True, False, True, True, False, True]
dist_angle_NACs:		 [True, True, True, True, True, False, True, True, False, True]
dist_angle_NACs_inverse:	 [False, False, False, False, False, True, False, False, True, False]
